<a href="https://colab.research.google.com/github/codened/DataStorm-4.0/blob/main/stormingRound/DataStorm_4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Path 
stormingRound/DataStorm_4_0.ipynb

# Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
rawHisTransDF=pd.read_csv('https://raw.githubusercontent.com/codened/DataStorm-4.0/main/stormingRound/Historical-transaction-data.csv?token=GHSAT0AAAAAACB2G7HINEMGWQOGRDX3O5SAZCHRPPA')
rawStoreInfDF=pd.read_csv('https://raw.githubusercontent.com/codened/DataStorm-4.0/main/stormingRound/Store-info.csv?token=GHSAT0AAAAAAB6LXUGKJUG26S55W43BMNDCZCHHHCQ')
# rawTestDF=pd.read_csv('https://raw.githubusercontent.com/codened/DataStorm-4.0/main/stormingRound/Testing-data.csv?token=GHSAT0AAAAAAB6LXUGLWEAXNZDDMU7C6VU2ZCG6V2A')

In [6]:
rawHisTransDF.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5


In [10]:
# convert the date string column to datetime
rawHisTransDF['transaction_date'] = pd.to_datetime(rawHisTransDF['transaction_date'])

In [12]:
# preprocess the data
rawHisTransDF['Date'] = pd.to_datetime(rawHisTransDF['transaction_date']) # convert the date column to datetime
rawHisTransDF['Month'] = rawHisTransDF['transaction_date'].dt.month # extract the month from the date
rawHisTransDF['Year'] = rawHisTransDF['transaction_date'].dt.year # extract the year from the date
rawHisTransDF['Total Sales'] = rawHisTransDF['item_price'] * rawHisTransDF['quantity_sold'] # calculate the total sales for each transaction

# create a pivot table to aggregate the total sales by store and month
pivot_table = pd.pivot_table(rawHisTransDF, values='Total Sales', index=['shop_id'], columns=['Year', 'Month'], aggfunc=np.sum, fill_value=0)

# convert the pivot table to a numpy array
X = pivot_table.values

In [19]:
rawHisTransDF.describe(include='all').T

<ipython-input-19-2a1e8cfc7a26>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rawHisTransDF.describe(include='all').T
<ipython-input-19-2a1e8cfc7a26>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rawHisTransDF.describe(include='all').T


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
item_description,438046,37,GINGER BEER 1.5L,59864,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
transaction_date,473974,62,2021-12-11 00:00:00+00:00,15125,2021-10-15 00:00:00+00:00,2021-12-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
invoice_id,467654.0,NaN,NaN,NaN,NaT,NaT,1996684.006321,1344594.276666,17.0,1032113.75,2032996.0,3032568.25,8331754.0
customer_id,473974,191636,RX33,95,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shop_id,473974,124,SHOP043,6631,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
item_price,473974.0,NaN,NaN,NaN,NaT,NaT,206.689734,166.764732,35.0,100.0,200.0,220.0,17400.0
quantity_sold,473974.0,NaN,NaN,NaN,NaT,NaT,1.925506,1.634535,-1.0,1.0,2.0,2.0,101.0
Date,473974,62,2021-12-11 00:00:00+00:00,15125,2021-10-15 00:00:00+00:00,2021-12-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Month,473974.0,NaN,NaN,NaN,NaT,NaT,11.017064,0.771991,10.0,10.0,11.0,12.0,12.0
Year,473974.0,NaN,NaN,NaN,NaT,NaT,2021.0,0.0,2021.0,2021.0,2021.0,2021.0,2021.0
